# HOW TO: Creando nuestro primer set de refactoriaciones.

Para refactorizar código, es necesario tener un conjunto de técnicas a nuestra disposición. En la mayoria de los casos, las refactorizaciones más comunes será extraer funcionalidades o variables a métodos o clases, renombrar variables o métodos para mejorar su legibilidad, o eliminar código duplicado. También es posible aplicar la técnica inversa, crear funciones y variables inline, para simplificar el código.

Para un mayor detalle de las técnicas de refactorización, Fowler (2019) menciona que:

> Extraction is all about giving names, and I often need to change the names as I learn.
> Change Function Declaration (124) changes names of functions; I also use that
> refactoring to add or remove a function’s arguments. For variables, I use Rename
> Variable (137), which relies on Encapsulate Variable (132). When changing function
> arguments, I often find it useful to combine a common clump of arguments into a
> single object with Introduce Parameter Object (140). (p. 121)



## Extract Function.

<small> _Inversion: Inline Function._ </small>

Consiste en extraer un fragmento de código a una función separada. Esto es útil para mejorar la legibilidad del código y para reutilizar el código en diferentes partes del programa. En software existen muchas maneras de definir exactamente en qué punto una función es demasiado larga. Algunos dirán que es por cantidad de líneas, otros por cantidad de responsabilidades. En cualquier caso es importante tener en cuenta que una función debe ser lo suficientemente pequeña como para ser fácilmente entendida y mantenida.

> If you have to spend effort looking at a fragment of code and figuring
> out hat its doing, then you should extract it into a function and name the function
> afterthe “wht.” Then, when you read it again, the purpose of the function leaps right
> out atyou, andmost of the time you won’t need to care about how the function fulfills
> its purose. (Fower, 2019, p. 122)

Para ejemplificar esta técnica usaremos un ejemplo.

La función `procesar_datos` actualmente realiza múltiples tareas sobre una lista de datos: limpia los valores de entrada, valida y transforma los datos numéricos, calcula métricas estadísticas, genera e imprime un reporte, guarda el resultado en un archivo bajo ciertas condiciones, muestra información de cada etapa y maneja posibles errores durante el procesamiento. Por ahora, se requiere analizar y refactorizar esta función, identificando y separando sus distintas responsabilidades para mejorar la claridad, mantenibilidad y reutilización del código, siguiendo las recomendaciones del capítulo 6 de Fowler sobre “Extract Function”.

```python
def procesar_datos(datos):
    # Limpiar datos
    datos_limpios = []
    for d in datos:
        if isinstance(d, str):
            d = d.strip()
        if d:
            datos_limpios.append(d)
    # Validar datos
    datos_validos = []
    for d in datos_limpios:
        if isinstance(d, int) and d > 0:
            datos_validos.append(d)
        elif isinstance(d, str) and d.isdigit():
            datos_validos.append(int(d))
    # Transformar datos (duplicar los valores pares)
    datos_transformados = []
    for d in datos_validos:
        if d % 2 == 0:
            datos_transformados.append(d * 2)
        else:
            datos_transformados.append(d)
    # Calcular métricas
    suma = 0
    maximo = None
    minimo = None
    for d in datos_transformados:
        suma += d
        if maximo is None or d > maximo:
            maximo = d
        if minimo is None or d < minimo:
            minimo = d
    promedio = suma / len(datos_transformados) if datos_transformados else 0
    # Generar reporte
    reporte = f"Total: {suma}, Máximo: {maximo}, Mínimo: {minimo}, Promedio: {promedio:.2f}, Elementos: {len(datos_transformados)}"
    print("==== REPORTE ====")
    print(reporte)
    # Guardar reporte en archivo si la suma es mayor a 100
    if suma > 100:
        with open("reporte.txt", "w") as f:
            f.write(reporte)
    # Logging adicional
    print("Datos originales:", datos)
    print("Datos limpios:", datos_limpios)
    print("Datos válidos:", datos_validos)
    print("Datos transformados:", datos_transformados)
    # Manejo de errores
    try:
        division = suma / (maximo - minimo)
        print("Suma dividida por rango:", division)
    except Exception as e:
        print("Error al calcular suma/rango:", e)
    # Mensaje final
    print("Procesamiento finalizado.")
```

Si el objetivo es mejorar la legibilidad de la función, lo primero que se podría hacer es identificar las distintas responsabilidades que tiene la función y extraerlas a funciones separadas. Por ejemplo, se podría extraer la limpieza de los valores de entrada a una función llamada `limpiar_datos` y la validación y transformación de los datos numéricos a una función llamada `validar_transformar_datos`. 

In [ ]:
def limpiar_datos(datos):
    # Elimina espacios en strings y filtra valores vacíos o None
    datos_limpios = []
    for d in datos:
        if isinstance(d, str):
            d = d.strip()
        if d is None or d == "":
            continue
        datos_limpios.append(d)
    return datos_limpios

def validar_transformar_datos(datos_limpios):
    # Valida: solo enteros positivos (convierte strings numéricos)
    datos_validos = []
    for d in datos_limpios:
        if isinstance(d, int):
            if d > 0:
                datos_validos.append(d)
        elif isinstance(d, str):
            if d.isdigit():
                val = int(d)
                if val > 0:
                    datos_validos.append(val)
        # Si no entra a ninguna condición, se ignora el valor
    # Transformación: duplicar valores pares
    datos_transformados = []
    for d in datos_validos:
        if d % 2 == 0:
            datos_transformados.append(d * 2)
        else:
            datos_transformados.append(d)
    return datos_transformados

def procesar_datos(datos):
    # 1) Limpieza
    datos_limpios = limpiar_datos(datos)
    # 2) Validación y transformación
    datos_transformados = validar_transformar_datos(datos_limpios)
    # 3) Calcular métricas
    suma = sum(datos_transformados)
    maximo = max(datos_transformados) if datos_transformados else None
    minimo = min(datos_transformados) if datos_transformados else None
    promedio = suma / len(datos_transformados) if datos_transformados else 0
    # 4) Generar reporte
    reporte = f"Total: {suma}, Máximo: {maximo}, Mínimo: {minimo}, Promedio: {promedio:.2f}, Elementos: {len(datos_transformados)}"
    print("==== REPORTE ====")
    print(reporte)
    # 5) Guardar si procede
    if suma > 100:
        try:
            with open("reporte.txt", "w", encoding="utf-8") as f:
                f.write(reporte)
        except Exception as e:
            print("Error al guardar reporte:", e)
    # 6) Logging de estados intermedios
    print("Datos originales:", datos)
    print("Datos limpios:", datos_limpios)
    print("Datos transformados:", datos_transformados)
    # 7) Manejo de posible error en cálculo adicional
    try:
        if maximo is None or minimo is None or maximo == minimo:
            raise ValueError("Rango inválido para división")
        division = suma / (maximo - minimo)
        print("Suma dividida por rango:", division)
    except Exception as e:
        print("Error al calcular suma/rango:", e)
    print("Procesamiento finalizado.")

# Ejemplo de uso
datos = [" 10 ", "20", 30, "", -5, "abc", 50, 3, "7", "  8", 0, None, "100"]
procesar_datos(datos)

En este sentido no importa que tan simple es la función, si el nombre de la nueva función reflejará claramente lo que hace entonces vale la pena extraerla. Sin embargo si la función ya tiene un nombre lo suficientemente claro entonces es una señal de que no es necesario extraerla. Sin embargo, algunas veces es dificil determinar si una función ya tiene un nombre lo suficientemente claro. Fowler (2019) explica que no es necesario que pienses en el mejor nombre posible para la nueva función en ese momento, a veces es válido extraer la función con un nombre provisional, trabajar con ella, darse cuenta que extraerla no mejora la legibilidad y devolverla a su estado original. Mientras se haya aprendido una lección jamás se está perdiendo el tiempo.

